# Ridge and Lasso Regression - Lab

## Introduction

In this lab, you'll practice your knowledge on Ridge and Lasso regression!

## Objectives

You will be able to:

- Use Lasso and ridge regression in Python
- Compare Lasso and Ridge with standard regression

## Housing Prices Data

Let's look at yet another house pricing data set.

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Housing_Prices/train.csv')

Look at df.info

In [42]:
# Your code here

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

We'll make a first selection of the data by removing some of the data with `dtype = object`, this way our first model only contains **continuous features**

Make sure to remove the SalesPrice column from the predictors (which you store in `X`), then replace missing inputs by the median per feature.

Store the target in `y`.

In [43]:
# Load necessary packages
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# remove "object"-type features and SalesPrice from `X`
X = df.select_dtypes(exclude=['object'])
X.drop(columns = 'SalePrice', inplace = True)

# Impute null values
for column in X:
    median = X[column].median()
    X[column].fillna(value = median, inplace=True)




# Create y
y = df["SalePrice"]


Look at the information of `X` again

In [44]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 37 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtFinSF2       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
LowQualFinSF     1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
BsmtHalfBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
F

## Let's use this data to perform a first naive linear regression model

Compute the R squared and the MSE for both train and test set.

In [45]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error

# Split in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Fit the model and print R2 and MSE for train and test
linreg = LinearRegression()
linreg.fit(X_train, y_train)

r2_train = linreg.score(X_train, y_train)
r2_test = linreg.score(X_test, y_test)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)
MSE_train = mean_squared_error(y_train, y_hat_train)
MSE_test = mean_squared_error(y_test, y_hat_test)


print(f"Train R^2: {r2_train}\nTest R^2: {r2_test}")
print(f"Train MSE: {MSE_train}\nTest MSE: {MSE_test}")







Train R^2: 0.8792812746143956
Test R^2: 0.45926221860708294
Train MSE: 768735679.7667004
Test MSE: 3266991413.4591403


## Normalize your data

We haven't normalized our data, let's create a new model that uses `preprocessing.scale` to scale our predictors!

In [79]:
from sklearn import preprocessing

# Scale the data and perform train test split
X_scaled = preprocessing.scale(X)

X_scaled_train, X_scaled_test, y_scaled_train, y_scaled_test = train_test_split(X, y)

Perform the same linear regression on this data and print out R-squared and MSE.

In [80]:
linreg_norm = LinearRegression()
linreg_norm.fit(X_scaled_train, y_scaled_train)

r2_scaled_train = linreg.score(X_scaled_train, y_scaled_train)
r2_scaled_test = linreg.score(X_scaled_test, y_scaled_test)

y_hat_scaled_train = linreg.predict(X_scaled_train)
y_hat_scaled_test = linreg.predict(X_scaled_test)
MSE_scaled_train = mean_squared_error(y_scaled_train, y_hat_scaled_train)
MSE_scaled_test = mean_squared_error(y_scaled_test, y_hat_scaled_test)




print(f"For Normalized Dataset:")
print(f"Train R^2: {r2_scaled_train}\nTest R^2: {r2_scaled_test}")
print(f"Train MSE : {MSE_scaled_train}\nTest MSE: {MSE_scaled_test}")


For Normalized Dataset:
Train R^2: 0.7844565996310823
Test R^2: 0.7592915168200037
Train MSE : 1415594785.6217234
Test MSE: 1326414095.8940718


## Include dummy variables

We haven't included dummy variables so far: let's use our "object" variables again and create dummies

In [84]:
# Create X_cat which contains only the categorical variables
X_cat = df.select_dtypes(include=['object'])
# (1460, 43) -> 43 rows


In [85]:
# Make dummies
X_cat = pd.get_dummies(X_cat)
# X_cat.shape = (1460, 252) -> 252 rows

Merge `x_cat` together with our scaled `X` so you have one big predictor dataframe.

In [86]:
# Your code here

X_all = pd.concat([pd.DataFrame(X_scaled), X_cat], axis = 1)

In [87]:
X_all.shape, X_scaled.shape, X_cat.shape

((1460, 289), (1460, 37), (1460, 252))

Perform the same linear regression on this data and print out R-squared and MSE.

In [102]:
y = df['SalePrice']

In [103]:
X_all.shape, y.shape

((1460, 289), (1460,))

In [100]:
X_all_train.shape, y_all_train.shape,X_all_test.shape, y_all_test.shape

((1095, 289), (1095,), (365, 289), (365,))

In [109]:
# Your code here

X_all_train, X_all_test, y_all_train, y_all_test = train_test_split(X_all, y)

linreg_all = LinearRegression()
linreg_all.fit(X_all_train, y_all_train)

r2_all_train = linreg_all.score(X_all_train, y_all_train)
r2_all_test = linreg_all.score(X_all_test, y_all_test)

y_hat_all_train = linreg_all.predict(X_all_train)
y_hat_all_test = linreg_all.predict(X_all_test)
MSE_all_train = mean_squared_error(y_all_train, y_hat_all_train)
MSE_all_test = mean_squared_error(y_all_test, y_hat_all_test)




print(f"For Normalized Dataset:")
print(f"Train R^2: {r2_all_train}\nTest R^2: {r2_all_test}")
print(f"Train MSE : {MSE_all_train}\nTest MSE: {MSE_all_test}")


For Normalized Dataset:
Train R^2: 0.9374012582923665
Test R^2: -2.3712046253831545e+19
Train MSE : 421833690.1805936
Test MSE: 1.1637426878415398e+29


Interpreting R squared values and MSE values:

training R squared is high, but the testing R squared is negative --> 
this means that there is overfitting
our predictions are very far off

Also, scale of the Testing MSE is orders of magnitude higher than that of the training



Notice the severe overfitting above; our training R squared is quite high, but the testing R squared is negative! Our predictions are far far off. Similarly, the scale of the Testing MSE is orders of magnitude higher then that of the training.

## Perform Ridge and Lasso regression

Use all the data (normalized features and dummy categorical variables) and perform Lasso and Ridge regression for both! Each time, look at R-squared and MSE.

## Lasso

With default parameter (alpha = 1)

In [129]:
from sklearn.linear_model import Lasso, Ridge

lasso_1 = Lasso(alpha=1) #Lasso is also known as the L1 norm.
lasso_1.fit(X_all_train, y_all_train)

print('Lass with Alpha = 1')

print('Training r^2:', lasso_1.score(X_all_train, y_all_train))
print('Testing r^2:', lasso_1.score(X_all_test, y_all_test))
print('Training MSE:', mean_squared_error(y_all_train, lasso_1.predict(X_all_train)))
print('Testing MSE:', mean_squared_error(y_all_test, lasso_1.predict(X_all_test)))

Lass with Alpha = 1
Training r^2: 0.9361725727422138
Testing r^2: -1.8123827568791495
Training MSE: 430113424.65374255
Testing MSE: 13802646273.941404


With a higher regularization parameter (alpha = 10)

In [130]:
lasso_10 = Lasso(alpha=10) #Lasso is also known as the L1 norm.
lasso_10.fit(X_all_train, y_all_train)
print('Lasso with Alpha = 10')

print('Training r^2:', lasso_10.score(X_all_train, y_all_train))
print('Testing r^2:', lasso_10.score(X_all_test, y_all_test))
print('Training MSE:', mean_squared_error(y_all_train, lasso_10.predict(X_all_train)))
print('Testing MSE:', mean_squared_error(y_all_test, lasso_10.predict(X_all_test)))



Lasso with Alpha = 10
Training r^2: 0.9337413211156573
Testing r^2: 0.8666114743521539
Training MSE: 446496882.49028987
Testing MSE: 654645827.285199


## Ridge

With default parameter (alpha = 1)

In [131]:
# Your code here

ridge_1 = Ridge(alpha=1) #Lasso is also known as the L1 norm.
ridge_1.fit(X_all_train, y_all_train)
print('Ridge with Alpha = 1')
print('Training r^2:', ridge_1.score(X_all_train, y_all_train))
print('Testing r^2:', ridge_1.score(X_all_test, y_all_test))
print('Training MSE:', mean_squared_error(y_all_train, ridge_1.predict(X_all_train)))
print('Testing MSE:', mean_squared_error(y_all_test, ridge_1.predict(X_all_test)))




Ridge with Alpha = 1
Training r^2: 0.9256742478900842
Testing r^2: 0.8138413284892662
Training MSE: 500858410.77138597
Testing MSE: 913631790.4824659


With default parameter (alpha = 10)

In [132]:
# Your code here
# Your code here

ridge_10 = Ridge(alpha=1) #Lasso is also known as the L1 norm.
ridge_10.fit(X_all_train, y_all_train)
print('Ridge with Alpha = 10')
print('Training r^2:', ridge_10.score(X_all_train, y_all_train))
print('Testing r^2:', ridge_10.score(X_all_test, y_all_test))
print('Training MSE:', mean_squared_error(y_all_train, ridge_10.predict(X_all_train)))
print('Testing MSE:', mean_squared_error(y_all_test, ridge_10.predict(X_all_test)))




Ridge with Alpha = 10
Training r^2: 0.9256742478900842
Testing r^2: 0.8138413284892662
Training MSE: 500858410.77138597
Testing MSE: 913631790.4824659


## Look at the metrics, what are your main conclusions?

Conclusions here

## Compare number of parameter estimates that are (very close to) 0 for Ridge and Lasso

In [141]:
# number of Ridge params almost zero
ridge_1_zeros = sum(abs(ridge_1.coef_) < 10**(-10))

In [142]:
ridge_10_zeros = sum(abs(ridge_10.coef_) < 10**(-10))

In [145]:

# number of Ridge params almost zero
lasso_1_zeros = sum(abs(lasso_1.coef_) < 10**(-10))

In [146]:
lasso_10_zeros = sum(abs(lasso_10.coef_) < 10**(-10))

In [148]:
ridge_1_zeros, ridge_10_zeros, lasso_1_zeros, lasso_10_zeros



(5, 5, 25, 65)

In [151]:
total = len(ridge_1.coef_
total

array([-7.17367489e+01, -4.51164976e+03, -1.33303907e+03,  1.96512974e+03,
        1.02695340e+04,  6.01766295e+03,  5.28492925e+03,  1.98423839e+03,
        1.57048529e+03,  5.16119305e+03,  1.52489062e+03, -3.27902716e+02,
        5.59627393e+03,  4.17732650e+03,  1.45781974e+04, -1.24922500e+03,
        1.50680406e+04,  8.95597532e+02, -3.02561851e+02,  2.88914184e+03,
        9.11667870e+02, -2.75498871e+03, -2.45319600e+03,  5.91629757e+03,
        3.57808484e+03, -5.09563364e+02,  7.49982931e+03,  7.22228420e+02,
        2.01148231e+03, -1.21816912e+02,  2.01673133e+02,  1.06260255e+03,
        1.76038084e+03, -2.67245312e+03, -6.67861960e+02, -1.12419066e+03,
       -3.57630171e+02, -2.66638901e+04,  1.41104010e+04,  2.98960823e+03,
        5.75833664e+03,  3.80554423e+03, -4.99428026e+03,  4.99428026e+03,
       -1.92592814e+03,  1.95249911e+03,  5.02311052e+02,  1.06075835e+04,
       -1.30510121e+04,  1.94111754e+03, -6.78160643e+03,  7.12965191e+03,
       -4.77985832e+03,  

Compare with the total length of the parameter space and draw conclusions!

## Summary

Great! You now know how to perform Lasso and Ridge regression.